# Frequent Patterns Analysis

In [35]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [36]:
df = pd.read_csv('data/tripadvisor_review.csv')
df.head()

,User ID,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10
0,User 1,0.930,1.800,2.290,0.620,0.800,2.420,3.190,2.790,1.820,2.420
1,User 2,1.020,2.200,2.660,0.640,1.420,3.180,3.210,2.630,1.860,2.320
2,User 3,1.220,0.800,0.540,0.530,0.240,1.540,3.180,2.800,1.310,2.500
3,User 4,0.450,1.800,0.290,0.570,0.460,1.520,3.180,2.960,1.570,2.860
4,User 5,0.510,1.200,1.180,0.570,1.540,2.020,3.180,2.780,1.180,2.540


## Preprocessing

In [26]:
# rename columns as their short names
cols_dict = {
	"Category 1": "AG", # art galleries
	"Category 2": "DC", # dance clubs
	"Category 3": "JB", # juice bars
	"Category 4": "RS", # restaurants
	"Category 5": "MU", # museums
	"Category 6": "RE", # resorts
	"Category 7": "PP", # parks/picnic spots
	"Category 8": "BE", # beaches
	"Category 9": "TH", # theaters
	"Category 10": "RI", # religious institutions
}

df.rename(columns=cols_dict, inplace=True)

In [27]:
# encode decimal review scores to binary values
threshold = 2
for col in df.columns[1:]:
	df[col] = df[col].apply(lambda x: 1 if x > threshold else 0)
df.head()

,User ID,AG,DC,JB,RS,MU,RE,PP,BE,TH,RI
0,User 1,0,0,1,0,0,1,1,1,0,1
1,User 2,0,1,1,0,0,1,1,1,0,1
2,User 3,0,0,0,0,0,0,1,1,0,1
3,User 4,0,0,0,0,0,0,1,1,0,1
4,User 5,0,0,0,0,0,1,1,1,0,1


In [28]:
# create a list of lists of reviews
reviews = []
for i, r in df.iterrows():
	review = [col for col in df.columns[1:] if r[col] == 1]
	reviews.append(review)
reviews[:5]

[['JB', 'RE', 'PP', 'BE', 'RI'],
 ['DC', 'JB', 'RE', 'PP', 'BE', 'RI'],
 ['PP', 'BE', 'RI'],
 ['PP', 'BE', 'RI'],
 ['RE', 'PP', 'BE', 'RI']]

In [29]:
# transform rewiews into a transaction matrix
te = TransactionEncoder()
te_ary = te.fit(reviews).transform(reviews)
review_df = pd.DataFrame(te_ary, columns=te.columns_)
review_df.head()

,AG,BE,DC,JB,MU,PP,RE,RI,RS,TH
0,False,True,False,True,False,True,True,True,False,False
1,False,True,True,True,False,True,True,True,False,False
2,False,True,False,False,False,True,False,True,False,False
3,False,True,False,False,False,True,False,True,False,False
4,False,True,False,False,False,True,True,True,False,False


## Apriori Algorithm

In [30]:
# calculate support for each item with min_sup = 0.1 using Apriori algorithm
min_sup = 0.1
frequent_itemsets_ap = apriori(review_df, min_support=min_sup, use_colnames=True)
frequent_itemsets_ap

,support,itemsets
0,1.000,(BE)
1,0.146,(JB)
2,1.000,(PP)
3,0.365,(RE)
4,1.000,(RI)
5,0.115,(TH)
6,0.146,"(BE, JB)"
7,1.000,"(BE, PP)"
8,0.365,"(BE, RE)"
9,1.000,"(BE, RI)"


In [31]:
# generate association rules
min_threshold = 0.5
rules = association_rules(frequent_itemsets_ap, metric="confidence", min_threshold=min_threshold)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(JB),(BE),0.146,1.000,0.146,1.000,1.000,0.000,inf,0.000
1,(BE),(PP),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
2,(PP),(BE),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
3,(RE),(BE),0.365,1.000,0.365,1.000,1.000,0.000,inf,0.000
4,(BE),(RI),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
5,(RI),(BE),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
6,(TH),(BE),0.115,1.000,0.115,1.000,1.000,0.000,inf,0.000
7,(JB),(PP),0.146,1.000,0.146,1.000,1.000,0.000,inf,0.000
8,(JB),(RI),0.146,1.000,0.146,1.000,1.000,0.000,inf,0.000
9,(RE),(PP),0.365,1.000,0.365,1.000,1.000,0.000,inf,0.000


In [32]:
# evaluate initial results
print(f"Number of rules: {len(rules)}")

Number of rules: 69


## FP-Growth Algorithm

In [33]:
# calculate support for each item with min_sup = 0.1 using FP-Growth algorithm
min_sup = 0.1
frequent_itemsets_fp = fpgrowth(review_df, min_support=min_sup, use_colnames=True)
frequent_itemsets_fp

,support,itemsets
0,1.000,(RI)
1,1.000,(PP)
2,1.000,(BE)
3,0.365,(RE)
4,0.146,(JB)
5,0.115,(TH)
6,1.000,"(RI, PP)"
7,1.000,"(BE, PP)"
8,1.000,"(BE, RI)"
9,1.000,"(BE, RI, PP)"


In [34]:
# generate association rules
min_threshold = 0.5
rules = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=min_threshold)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(RI),(PP),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
1,(PP),(RI),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
2,(BE),(PP),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
3,(PP),(BE),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
4,(BE),(RI),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
5,(RI),(BE),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
6,"(BE, RI)",(PP),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
7,"(BE, PP)",(RI),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
8,"(RI, PP)",(BE),1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
9,(BE),"(RI, PP)",1.000,1.000,1.000,1.000,1.000,0.000,inf,0.000
